### 1. Initial Configuration

#### 1.1 Imports & Config

In [1]:
# data analysis
import numpy as np
import pandas as pd
import xlwings as xw

# plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as po
import plotly.figure_factory as ff

# colors
import colorlover as cl
from IPython.display import HTML

import requests

In [2]:
%matplotlib inline

In [3]:
po.init_notebook_mode(connected=True)

#### 1.2 Global Functions

In [4]:
def flatten_list(lst):
    flat_list = []
    
    for val in lst:
        if isinstance(val, list):
            flat_list += flatten_list(val)
        else:
            flat_list.append(val)
    
    return flat_list

In [5]:
def sum_row_vals(series):
    out = ''
    
    for i in range(len(series)):
        out += str(series[i]) + '  '
    
    return out

In [6]:
def get_color_scale(index):
    color_scale = []
    
    try:
        trace_cnt = len(index.labels[0])
    except AttributeError:
        trace_cnt = len(index)
    
    while trace_cnt > 0:
        if trace_cnt > 12:
            color_scale += cl.scales['12']['qual']['Paired']
            trace_cnt -= 12
        elif trace_cnt > 2:
            color_scale += cl.scales[str(trace_cnt)]['qual']['Paired']
            trace_cnt = 0
        else:
            color_scale += cl.scales['3']['qual']['Paired'][:trace_cnt]
            trace_cnt = 0
    
    return color_scale

#### 1.3 Code Examples

In [7]:
HTML(cl.to_html(cl.scales))

### 2. Data Preparation (ETL)

#### 2.1 State Level Population Estimates, 1900-2016

In [8]:
def read_census_data_1900_1969(url, header=0, nrows=0, years=None):
    # Read in raw data
    df = pd.read_fwf(
        url,
        header = header,
        nrows = nrows
    )

    # Pull Column Names
    new_col_nms = flatten_list(['State'] + list(df.columns.str.split('\s+')))
    
    # Flatten DF into single columns
    if df.shape[1] > 1:
        df = pd.DataFrame(df.apply(sum_row_vals, axis=1))
    
    # Split data into multiple columns
    df = df.iloc[:, 0].str.strip().str.split('\s{2,}', expand=True)

    # Rename Columns
    df = df.rename(columns = {col: new_col for col, new_col in zip(df.columns, new_col_nms)})
    
    # Drop Aggregate Columns
    non_states = df['State'].isin(['U.S.', 'Northeast', 'North Central', 'South', 'West', 'Midwest', 'REGIONS:', 'STATES:'])
    df = df.loc[-non_states].reset_index(drop = True)
    
    # Drop null rows & columns
    df = df.dropna(how='any').reset_index(drop = True)
    
    # Conver String Numbers to Integers
    f = (lambda col: col.str.replace(',', ''))
    converted_data = df.iloc[:, 1:].apply(f, axis = 1).astype(np.int64)
    df = df.assign(**{col: converted_data[col].values for col in converted_data.columns})
    
    # Flag duplicate columns
    new_cols = [df.columns[i] if df.columns[i] != df.columns[i+1] else 'dupe' for i in range(len(df.columns)-1)] + [df.columns[-1]]
    df.columns = new_cols
    
    # Logic to drop irrelevant columns, and duplicate columns (take the second one)
    if years is not None:
        accepted_cols = ['State'] + [str(yr) for yr in range(years[0], years[1]+1)]
        df = df.loc[:, df.columns.isin(accepted_cols)]
    
    # Multiply by 1000 if file data is represented "in thousands"
    response = requests.get(url)
    if 'in thousands' in response.text[:500].lower():
        thousands = df.loc[:, df.dtypes == np.int64] * 1000
        df = df.assign(**{col: thousands[col].values for col in thousands.columns})
    
    return df

In [9]:
def read_census_data_1970_1989(url, header=0, nrows=0, years=None):
    # Read in raw data
    df = pd.read_fwf(
        url,
        header = header,
        nrows = nrows
    )

    # Pull Column Names
    new_col_nms = pd.Series(flatten_list(df.columns.str.split('\s+'))) \
                    .str.replace('.*/', '') \
                    .str.replace('cen', '') \
                    .str.replace('St', 'State')

    new_col_nms = new_col_nms.where(
        -new_col_nms.str.isnumeric(),
        lambda x: '19' + x
    ).tolist()

    # Flatten DF into single columns
    if df.shape[1] > 1:
        df = pd.DataFrame(df.apply(sum_row_vals, axis=1))

    # # Split data into multiple columns
    df = df.iloc[:, 0].str.strip().str.split('\s+', expand=True)

    # # Rename Columns
    if df.shape[1] > len(new_col_nms):
        new_col_nms = (['State'] * (df.shape[1] - len(new_col_nms))) + new_col_nms

    df = df.rename(columns = {col: new_col for col, new_col in zip(df.columns, new_col_nms)})

    # Flag duplicate columns
    new_cols = [df.columns[i] if df.columns[i] != df.columns[i+1] else 'dupe' for i in range(len(df.columns)-1)] + [df.columns[-1]]
    df.columns = new_cols

    # Drop Aggregate Columns
    non_states = df['State'].isin(['US'])
    df = df.loc[-non_states].reset_index(drop = True)

    # # Drop null rows & columns
    # df = df.dropna(how='any').reset_index(drop = True)

    # Convert String Numbers to Integers
    f = (lambda col: col.str.replace(',', ''))
    converted_data = df.loc[:, df.columns.str.isnumeric()].apply(f, axis = 1).astype(np.int64)
    df = df.assign(**{col: converted_data[col].values for col in converted_data.columns})

    # Logic to drop irrelevant columns, and duplicate columns (take the second one)
    if years is not None:
        accepted_cols = ['State'] + [str(yr) for yr in range(years[0], years[1]+1)]
        df = df.loc[:, df.columns.isin(accepted_cols)]
        
    # Multiply by 1000 if file data is represented "in thousands"
    response = requests.get(url)
    if 'in thousands' in response.text[:500].lower():
        thousands = df.loc[:, df.dtypes == np.int64] * 1000
        df = df.assign(**{col: thousands[col].values for col in thousands.columns})
    
    return df

In [10]:
parameters = pd.DataFrame({
    'years' : [
        (1900, 1905), (1906, 1909), 
        (1910, 1915), (1916, 1919), 
        (1920, 1925), (1926, 1929),
        (1930, 1935), (1936, 1939),
        (1940, 1945), (1946, 1949),
        (1950, 1954), (1955, 1959),
        (1960, 1964), (1965, 1969),
        (1970, 1975), (1976, 1979),
        (1980, 1984), (1985, 1989)
    ],
    'header': [11, 66, 11, 66, 11, 66, 11, 66,  9, 64, 16, 81, 11, 69,  9, 62, 6, 62],
    'nrows' : [56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 61, 61, 61, 61, 51, 51, 52, 52],
    'url'   : ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st0009ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st1019ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st2029ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st3039ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st4049ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st5060ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st6070ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st7080ts.txt'] * 2 + \
                ['https://www2.census.gov/programs-surveys/popest/tables/1980-1990/state/asrh/st8090ts.txt'] * 2,
    'func'  : ([read_census_data_1900_1969] * 14) + ([read_census_data_1970_1989] * 4)
}, columns = ['years', 'header', 'nrows', 'url', 'func'])

In [11]:
# Read Individual Row 
# row = 2
# df = parameters.iloc[row].func(
#     url = parameters.iloc[row].url,
#     header = parameters.iloc[row].header,
#     nrows = parameters.iloc[row].nrows,
#     years = parameters.iloc[row].years
# )
# print(df.shape)
# df

In [12]:
%%time
# Read in Payments Data
pop = pd.concat(
    (
        parms.func(
            url = parms.url,
            header = parms.header,
            nrows = parms.nrows,
            years = parms.years
        ).melt(
            id_vars='State', 
            var_name='Year', 
            value_name='Population'
        )
        for _, parms in parameters.iterrows()
    ),
    ignore_index=True
)

Wall time: 9.59 s


#### 2.1 Projected Future Health Expenditures

In [17]:
pop_by_ST_and_YR = pop.pivot_table(
    values = 'Population', 
    index = 'State',
    columns = 'Year',
    aggfunc = np.sum,
    fill_value = 0
)

In [19]:
def stacked_line_plot(df, normalize=False, cumulative_text=False, color_scale=None, nbr_form=None, sort_desc=True, inline=False):
    # set defaults
    df = (df if not normalize else (df / df.sum(axis=0)))
    color_scale = (color_scale if color_scale is not None else get_color_scale(df.index))
    
    if nbr_form is None:    
        if normalize:
            nbr_form = (lambda x: '{:.1f}%'.format(x * 100))
        else:
            nbr_form = (lambda x: '${:.1f}'.format(x / 1000))

    # Data Preparation
    if sort_desc:
        df = df.sort_values(by = df.columns[-1], ascending = False)
    
    y = df.cumsum()
    
    if cumulative_text:
        y_txt = y.applymap(nbr_form)
    else:
        y_txt = df.applymap(nbr_form)
    
    # Build Traces with list comprehension
    traces = [
        go.Scatter(
            x = data[1].index,
            y = data[1].values,
            text = data_txt[1].values,
            hoverinfo = 'x+text',
            mode = 'lines',
            line = dict(
                width = 1,
                color = color
            ),
            fill = 'tonexty',
            name = data[0]
        ) for data, data_txt, color in zip(y.iterrows(), y_txt.iterrows(), color_scale)
    ]

    fig = go.Figure(data=traces)
    
    if inline:
        po.iplot(fig, filename='my-stacked-area-plot-hover')
    else:
        po.plot(fig, filename='my-stacked-area-plot-hover')

In [20]:
 stacked_line_plot(
     pop_by_ST_and_YR.loc[pop_by_ST_and_YR.iloc[:, -1].nlargest(10).index],
     normalize = False,
     nbr_form = (lambda x: '{:.2f} M'.format(x / 1000000)),
     inline = True
)

#### 2.2 Population By Gender, State, and Age

### 3. Visualizations

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['state'] + '<br>' +\
    'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
    'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
    'Wheat '+df['wheat']+' Corn '+df['corn']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['total exports'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(0,0,0)',
                width = 0.25
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = False,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
po.iplot( fig, filename='d3-cloropleth-map' )

In [21]:
colors = cl.scales['7']['seq']['Reds']
# vals = np.linspace(0, 1, num = len(colors))
vals = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 1]
scale = [[val, color] for color, val in zip(colors, vals)]
scale

[[0, 'rgb(254,229,217)'],
 [0.1, 'rgb(252,187,161)'],
 [0.2, 'rgb(252,146,114)'],
 [0.3, 'rgb(251,106,74)'],
 [0.4, 'rgb(239,59,44)'],
 [0.5, 'rgb(203,24,29)'],
 [0.6, 'rgb(153,0,13)']]

In [22]:
colors = cl.scales['9']['seq']['Blues']
# vals = np.linspace(0, 1, num = len(colors))
vals = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 1]
scale = [[val, color] for color, val in zip(colors, vals)]

data = [ dict(
        type='choropleth',
        colorscale = scale,
        autocolorscale = False,
        locations = pop_by_ST_and_YR.index,
        z = pop_by_ST_and_YR['1989'],
        locationmode = 'USA-states',
        text = pop_by_ST_and_YR['1989'].apply(lambda x: '{:.2f} M'.format(x / 1000000)),
        marker = dict(
            line = dict (
                color = 'rgb(0,0,0)',
                width = 0.25
            ) ),
        colorbar = dict(
            title = "Population in Millions")
        ) ]

layout = dict(
    title = 'US Population by State)',
    geo = dict(
        scope='usa',
    ),
)
    
fig = dict( data=data, layout=layout )
po.iplot( fig, filename='d3-cloropleth-map' )

In [23]:
colors = cl.scales['9']['seq']['Blues']
# vals = np.linspace(0, 1, num = len(colors))
vals = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 1]
scale = list(zip(vals, colors))

data = [dict(
    visible = False,
    type = 'choropleth',
    colorscale = scale,
    autocolorscale = False,
    locations = pop_by_ST_and_YR.index,
    z = pop_by_ST_and_YR[year],
    locationmode = 'USA-states',
    text = pop_by_ST_and_YR[year].apply(lambda x: '{:.2f} M'.format(x / 1000000)),
    marker = dict(
        
        line = dict(
            color = 'rgb(0,0,0)',
            width = 0.25
        )
    ),
    colorbar = dict(
        title = "Population in Millions, " + str(year)
    )
) for year in pop_by_ST_and_YR.columns]
data[0]['visible'] = True

steps = []
for i in range(len(pop_by_ST_and_YR.columns)):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(pop_by_ST_and_YR.columns)],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 0,
    currentvalue = {"prefix": "Year: "},
    pad = {"t": 50},
    steps = steps
)]

layout = dict(
    title = 'US Population by State',
    geo = dict(
        scope='usa',
    ),
    sliders = sliders
)
    
fig = dict( data=data, layout=layout )
po.iplot( fig, filename='d3-cloropleth-map' )